In [2]:
import io
import re
import csv
import pandas as pd
import win32com.client
from docx import Document

In [3]:
PathQ = "I.docx"

In [4]:
prov = open('../provinsi.txt', 'r').read().lower()

In [5]:
prov = prov.split('\n')

In [6]:
prov = set(prov[:len(prov)-1])

In [7]:
docQ = Document(open(PathQ, 'rb'))

In [8]:
i = 0
par_1 = []
for idx, para in enumerate(docQ.paragraphs):
    if (para.text.lower() not in prov) and (len(para.text.strip(' \n\r\t')) != 0):
        tes = i, para
        par_1.append(tes)
        i = i+1

In [9]:
i = 0
y = 0
par2 = []
kab = []
kab_bold = []
for idx, para in par_1:
#     print(para)
    if idx in kab_bold:
        continue
    temp = {}
#     if i == 0:
#         kab[i]['end_pos'] = 0
    if idx == len(par_1)-1:
        kab[i-1]['end_pos'] = idx
    for r in para.runs:
        if r.font.size == 114300:
            temp_kabkot = para.text
            cek_tit = True
            xx = idx + 1
            if(xx == len(par_1)):
                cek_tit = False
            while cek_tit:
                temp_kabkot2 = [run.text for run in par_1[xx][1].runs if (run.bold) and (run.font.size == 114300)]
                if len(temp_kabkot2) > 0:
                    kab_bold.append(xx)
                    temp_kabkot = temp_kabkot +' '+temp_kabkot2[0]
                    xx = xx + 1
                elif(xx == len(par_1)):
                    cek_tit = False
                else:
                    cek_tit = False
            
#             print(temp_kabkot)
#             dist = re.search(r'KOTA |KAB. ',temp_kabkot).group()
#             nama = re.sub(r'KOTA |KAB. ','', temp_kabkot)
#             nama = re.sub(r'\s+', '',nama)
            temp['text'] = temp_kabkot
            if (i != 0) and (idx != len(par_1)-1):
                kab[i-1]['end_pos'] = idx-2
            kab.append(temp)
            i = i+1
        else:
            temp2 = y, para
            par2.append(temp2)
            y = y+1

In [10]:
kab = pd.DataFrame(kab)

In [11]:
kab.head()

,end_pos,text
0,5,KAB. ACEH TENGGARA
1,10,KAB. ACEH TENGAH
2,27,KAB. ACEH BARAT
3,43,KAB. ACEH BESAR
4,56,KAB. BIREUEN


In [12]:
len(par2)

78831

In [13]:
par = par2
for idx, para in par:
    for run in para.runs:
        if (run.font.size == 101600) and (run.bold):
            run.bold = None
#             run.font.size = 76200

In [14]:
batas = 0
pars = []
title_bold = []
kat_bold = []
for idx, para in par:
    if (idx in title_bold) :
        continue
    paratemp = []
    for run in para.runs:
        if run.bold:
#             print(run.text)
            temptit = [run.text for run in para.runs if run.bold][0]
            cek_tit = True
            xx = idx + 1
            if(xx == len(par)):
                cek_tit = False
            while cek_tit:
                temptit2 = [run.text for run in par[xx][1].runs if run.bold]
                if len(temptit2) > 0:
                    title_bold.append(xx)
                    temptit = temptit +' '+temptit2[0]
#                     print(temptit)
                    xx = xx + 1
                    if(xx == len(par)):
                        cek_tit = False
                else:
                    cek_tit = False
            paratemp.append(temptit+' [NAMA_PERUSAHAAN]')
#             print(temptit)
#             print(temptit+' [NAMA_PERUSAHAAN]')
#             print(idx)
            for x in range(batas, idx):
                if x in kat_bold:
                    continue
                for run in par[x][1].runs:
                    if len(run.text.strip(' \n\r\t')) != 0:
                        if run.bold:
                            continue
                        elif run.font.size == 101600:
                            temp_kat = run.text
#                             print(temp_kat)
                            xxx = x + 1
                            cek_kat = True
                            if xxx < batas:
                                cek_kat = False
                            while cek_kat:
                                for run in par[xxx][1].runs:
                                    if (run.font.size == 101600):
                                        temp_kat = temp_kat +' '+par[xxx][1].text
                                        kat_bold.append(xxx)
                                        xxx = xxx + 1
                                        if xx >= batas:
                                            cek_kat = False
                                    else:
                                        cek_kat = False
                            paratemp.append(temp_kat)
                        else:
                            temp_alamat1 = par[x][1].text
                            paratemp.append(temp_alamat1)
            
            pars.append(paratemp)
            batas = xx

In [15]:
pars[:20]
# kab[kab.end_pos >= 9]

[['SARTIKA HOTEL [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '082299077118',
  'hotel.sartika@yahoo.com',
  'JL RAJA BINTANG NO 3, MBARUNG, KEC.',
  'BABUSSALAM 24651'],
 ['RENGGALI HOTEL [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '0643-21144',
  'JL DEDALU - BINTANG, ONE-ONE, KEC. LUT TAWAR'],
 ['MEULIGOE GAJAH PUTEH, PT [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '0655-7007171',
  'JL ISKANDAR MUDA NO 36, UJUNG KALAK, KEC.',
  'JOHAN PAHLAWAN 23613'],
 ['TIARA HOTEL [NAMA_PERUSAHAAN]',
  'JL TEUKU UMAR DSN JEUMPA, UJUNG KALAK, KEC.',
  'JOHAN PAHLAWAN 23613'],
 ['KFC CABANG MEULABOH [NAMA_PERUSAHAAN]',
  '56 PENYEDIAAN MAKANAN DAN MINUMAN',
  'kfcku.com',
  '081905332573',
  'mla@ffi.co.id.',
  'JL MANEK R00 NO 135-138, DRIEN RAMPAK, KEC.',
  'JOHAN PAHLAWAN 23617'],
 ['THE PADE HOTEL [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '0651-49999',
  '0651-47999',
  'JL SOEKARNO HATTA NO 1, DAROY KAMEU, KEC.',
  'DARUL IMARAH 23352'],
 ['BAKSO NANGGROE <SUWIRNA>

In [16]:
# menggabungkan alamat
par_temp = []
for idx, p in enumerate(pars):
    parss = []
    batas = len(p)-1
    batas_bawah = []
    for idxx, x in enumerate(p):
        if idxx in batas_bawah:
            continue
        tempAlamat = ''
        if idxx <= batas:
            if re.search(r'( \[NAMA_PERUSAHAAN\])', x):
                parss.append(x)
            elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                if idxx == len(p)-1:
                    tempAlamat = x
                else:
                    xxx = idxx
                    while (xxx <= len(p)-1):
                        if (re.search(r'[^\-]\b[\d]{5}\b', p[xxx])):
                            batas_bawah.append(xxx)
                            tempAlamat = tempAlamat + ' '+ p[xxx]
                            xxx = xxx + 1
                        elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                            batas_bawah.append(xxx)
                            tempAlamat = tempAlamat + ' '+ p[xxx]
                            xxx = xxx + 1
                if len(kab) > 0:
                    if (re.search(r'[^\-]\b[\d]{5}\b', tempAlamat)) and (re.search(r'Ext: \b[\d]{5}\b', tempAlamat) == None):
                        kode_pos = re.search(r'[^\-]\b[\d]{5}\b', tempAlamat).group().strip()
                        alamat = re.sub(r'[^\-]\b[\d]{5}\b', '',  tempAlamat)
                        temp_alamat1 = alamat + ', '+ kab[kab.end_pos >= idx].head(1).iloc[0].text + ' '+kode_pos
                    else:
                        if (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', tempAlamat)):
                            temp_alamat1 = tempAlamat + ', '+ kab[kab.end_pos >= idx].head(1).iloc[0].text
                        else:
                            temp_alamat1 = tempAlamat
                else:
                    temp_alamat1 = tempAlamat
                parss.append(temp_alamat1.strip(' \n\r\t'))
            else:
                parss.append(x)
    par_temp.append(parss)

In [17]:
par_temp[:5]

[['SARTIKA HOTEL [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '082299077118',
  'hotel.sartika@yahoo.com',
  'JL RAJA BINTANG NO 3, MBARUNG, KEC. BABUSSALAM, KAB. ACEH TENGGARA 24651'],
 ['RENGGALI HOTEL [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '0643-21144',
  'JL DEDALU - BINTANG, ONE-ONE, KEC. LUT TAWAR, KAB. ACEH TENGGARA'],
 ['MEULIGOE GAJAH PUTEH, PT [NAMA_PERUSAHAAN]',
  '55 PENYEDIAAN AKOMODASI',
  '0655-7007171',
  'JL ISKANDAR MUDA NO 36, UJUNG KALAK, KEC. JOHAN PAHLAWAN, KAB. ACEH TENGGARA 23613'],
 ['TIARA HOTEL [NAMA_PERUSAHAAN]',
  'JL TEUKU UMAR DSN JEUMPA, UJUNG KALAK, KEC. JOHAN PAHLAWAN, KAB. ACEH TENGGARA 23613'],
 ['KFC CABANG MEULABOH [NAMA_PERUSAHAAN]',
  '56 PENYEDIAAN MAKANAN DAN MINUMAN',
  'kfcku.com',
  '081905332573',
  'mla@ffi.co.id.',
  'JL MANEK R00 NO 135-138, DRIEN RAMPAK, KEC. JOHAN PAHLAWAN, KAB. ACEH TENGGARA 23617']]

In [24]:
# Buat DF
data = []
tempTel1 = ''
tempTel2 = ''
tempTel3 = ''  
tempEmail = ''
tempWeb = ''
for idx, line in enumerate(par_temp):

    row = {'Kategori': '',
           'Telp1': '', 
           'Telp2': '', 
           'Telp3': '',
           'Email': '',
           'Website': '',
           'Alamat': '',
           'Nama Perusahaan': '',
           'Kode Kategori': '',
           'Status Badan Usaha': ''}

    for text in line:
        if(re.search(r'[0-9]{3,}-\d{5,}|^08|\+62|\d{3}-62-\d{2,3}-\d{5,}|--\d{5,}|\d{6,}', text)):
            if(tempTel1 == ''):
                tempTel1 = text
                row['Telp1'] = tempTel1
            elif(tempTel2 == ''):
                tempTel2 = text
                row['Telp2'] = tempTel2
            else:
                tempTel3 = text
                row['Telp3'] = tempTel3
        elif(re.search(r'( \[NAMA_PERUSAHAAN\])', text)):
#             print(text)
            text = re.sub(r'( \[NAMA_PERUSAHAAN\])', '', text)
            if (re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text)):
                row['Nama Perusahaan'] = re.sub(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)','',text)
                row['Status Badan Usaha'] = re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text).group().replace(',','').strip(' \n\r\t')
            else:
                row['Nama Perusahaan'] = text
                if(re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bGMBH\b)|(\bBUT\b)|(\bS\.P\.A\b)',text)):
                    row['Status Badan Usaha'] = re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bBUT\b)|(\bGMBH\b)|(\bS\.P\.A\b)',text).group()
                elif(re.search(r'\bLIMITED\b',text)):
                    row['Status Badan Usaha'] = 'LTD'
                elif(re.search(r'\b KAB \b|\b KOTA \b',text)):
                    row['Status Badan Usaha'] = 'PD'
                else:
                    row['Status Badan Usaha'] = 'None'
        elif(re.search(r'.*@.*\.[a-zA-Z]{2,3}\.?', text)) or (re.search(r'.*@.*', text)):
            tempEmail = text
            row['Email'] = tempEmail
        elif(re.search(r'(^www.)|(^http)|(.com$)|(.co.id$)|(.net$)', text)):
            tempWeb = text
            row['Website'] = tempWeb
        elif(re.search(r'^[0-9]{2}[a-zA-Z\s\-]+( - .*)?$', text)):
            row['Kategori'] = re.sub(r'\d+ | - .*','', text)
            row['Kode Kategori'] = re.sub(r' [a-zA-Z].*','', text)
        elif(re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', text)):
            row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
        else:
            row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
            
    if row['Kategori'] == '':
        row['Kategori'] = data[idx-1]['Kategori']
        row['Kode Kategori'] = data[idx-1]['Kode Kategori']
#         print(row['Kategori'])
    data.append(row)
    tempTel1 = ''
    tempTel2 = ''
    tempTel3 = ''  
    tempEmail = ''
    tempWeb = ''

In [25]:
df = pd.DataFrame(data)

In [27]:
df.head()

,Alamat,Email,Kategori,Kode Kategori,Nama Perusahaan,Status Badan Usaha,Telp1,Telp2,Telp3,Website
0,"JL RAJA BINTANG NO 3, MBARUNG, KEC. BABUSSALAM...",hotel.sartika@yahoo.com,PENYEDIAAN AKOMODASI,55,SARTIKA HOTEL,None,082299077118,,,
1,"JL DEDALU - BINTANG, ONE-ONE, KEC. LUT TAWAR, ...",,PENYEDIAAN AKOMODASI,55,RENGGALI HOTEL,None,0643-21144,,,
2,"JL ISKANDAR MUDA NO 36, UJUNG KALAK, KEC. JOHA...",,PENYEDIAAN AKOMODASI,55,MEULIGOE GAJAH PUTEH,PT,0655-7007171,,,
3,"JL TEUKU UMAR DSN JEUMPA, UJUNG KALAK, KEC. JO...",,PENYEDIAAN AKOMODASI,55,TIARA HOTEL,None,,,,
4,"JL MANEK R00 NO 135-138, DRIEN RAMPAK, KEC. JO...",mla@ffi.co.id.,PENYEDIAAN MAKANAN DAN MINUMAN,56,KFC CABANG MEULABOH,None,081905332573,,,kfcku.com


In [28]:
df.to_csv('I.csv')